# Bay Wheels Operation

In [ ]:
import pandas as pd

In [ ]:
path = 'https://raw.githubusercontent.com/MCanela-1954/Data/main/'

In [ ]:
rides1 = pd.read_csv(path + 'bay_rides-1.csv.zip')
rides2 = pd.read_csv(path + 'bay_rides-2.csv.zip')
rides3 = pd.read_csv(path + 'bay_rides-3.csv.zip')
rides4 = pd.read_csv(path + 'bay_rides-4.csv.zip')
rides5 = pd.read_csv(path + 'bay_rides-5.csv.zip')

In [ ]:
rides = pd.concat([rides1, rides2, rides3, rides4, rides5])

In [ ]:
rides.info()

In [ ]:
rides.head()

We group by month to see the monthly variation in demand

In [ ]:
rides['month'] = rides['start_time'].str[:-11] + '01 00:00:00'
rides.head()

In [ ]:
rides['month'] = rides['month'].astype('datetime64[ns]')
rides.info()

In [ ]:
rides['electric'] = (rides['bike_type'] == 'electric')
rides['classic'] = (rides['bike_type'] == 'classic')

We sum the data to group it by month

### Question 1 - Analysis of the Variation in Demand Between Classic and Electric Bikes

In [ ]:
df1 = rides[['month', 'electric', 'classic']].groupby(by='month').sum()
df1.head()

Indexed using month

In [ ]:
df1.index

In [ ]:
df1.index.name = None

In [ ]:
df1['total'] = df1['classic'] + df1['electric']

The result of grouping by month

In [ ]:
df1['month'] = df1.index
df1.head()

In [ ]:
df1[['electric', 'classic', 'month']].groupby('month').sum().plot(figsize=(7,5),
    title='Figure 1. Trend of total demand based on bike type', color=['0.4', '0.7'])

In [ ]:
df1.tail(8)

As we can see, the trend of both electric and classic bikes are the same, with both reaching their own respective peaks and lows depending on the seasonality. However, in the end 2023, there is a significant dip for the electric bike demand compared to the classic bike demand, which continued to record increase in demand unlike electric, which started to record decrease since November 2023.

Data in the first quarter of 2024 can greatly help see whether it is just an anomaly if the trend goes back to the trend that was seen in previous years.

Let us go further than the initial question by looking at other statistics. Looking only at the trend does not show us the change in share between the two products in percentage. Let us see how the percentage has changed over the year.

In [ ]:
# Extract the year
df1['year'] = df1['month'].dt.year

# Group by year and bike type and sum
yearly_bike_type = df1.groupby('year')[['electric', 'classic']].sum()

# Calculate the total rides per year
yearly_bike_type['total'] = yearly_bike_type['electric'] + yearly_bike_type['classic']

# Calculate the percentage share
yearly_bike_type['electric_percentage'] = (yearly_bike_type['electric'] / yearly_bike_type['total']) * 100
yearly_bike_type['classic_percentage'] = (yearly_bike_type['classic'] / yearly_bike_type['total']) * 100

# Display the result
print(yearly_bike_type[['electric_percentage', 'classic_percentage']])

In terms of share, we can conclude that classic has increased in share between the two. However, this can lead us into believing that the total demand for electric is sinking, and it is being taken over by classic. Therefore, we should calculate the CAGR (compounded annual growth rate) to see the growth of each product individually.

In [ ]:
# Assuming 'yearly_bike_type' DataFrame is already created and contains 'electric' and 'classic' rides per year

# Get the first and last year in your data
first_year = yearly_bike_type.index.min()
last_year = yearly_bike_type.index.max()

# Get the ride counts for the first and last full year for electric and classic
beginning_electric = yearly_bike_type.loc[first_year, 'electric']
ending_electric = yearly_bike_type.loc[last_year, 'electric']

beginning_classic = yearly_bike_type.loc[first_year, 'classic']
ending_classic = yearly_bike_type.loc[last_year, 'classic']

# Determine the number of years
number_of_years = last_year - first_year

# Calculate CAGR for electric bikes
cagr_electric = (ending_electric / beginning_electric)**(1 / number_of_years) - 1

# Calculate CAGR for classic bikes
cagr_classic = (ending_classic / beginning_classic)**(1 / number_of_years) - 1

print(f"The Compound Annual Growth Rate (CAGR) for electric rides from {first_year} to {last_year} is: {cagr_electric:.2%}")
print(f"The Compound Annual Growth Rate (CAGR) for classic rides from {first_year} to {last_year} is: {cagr_classic:.2%}")

By calculating the CAGR, we now know that both products are growing in demand. However, since classic had lower starting point, it is generally easier for products with lower amount of demand in the beginning to get a higher growth.

### Question 2 - Dockless Sharing Trend Analysis

In [ ]:
rides['dockless'] = rides['start_station_id'].isnull() | rides['end_station_id'].isnull()
rides['reg_electric'] = (rides['electric'] & ((rides['start_station_id'].isnull() | rides['end_station_id'].isnull()) == False))
rides.head()

In [ ]:
df2 = rides[['month', 'classic', 'electric', 'dockless', 'reg_electric']].groupby(by='month').sum()
df2.head()

We evaluate the truthfulness that only electrics can be dockless, and we get the difference between all electric and dockless electric

In [ ]:
electric = rides['electric'].sum()
dockless_electric = (rides['dockless'] & rides['electric']).sum()
docking_electric = (rides['electric'] & (rides['dockless'] == False)).sum()
classic = rides['classic'].sum()
dockless_classic = (rides['dockless'] & rides['classic']).sum()

print("Electric Total          : %10d" % electric)
print("Dockless Electric Total : %10d" % dockless_electric)
print("Docking Electric Total  : %10d" % docking_electric)
print("Classic Total           : %10d" % classic)
print("Dockless Classic Total  : %10d" % dockless_classic)

We have proved that only electrics can be dockless

In [ ]:
df2.index.name = None

In [ ]:
df2['month'] = df2.index
df2.head()

In [ ]:
df2[['electric', 'dockless', 'reg_electric', 'month']].groupby('month').sum().plot(figsize=(7,5),
    title='Figure 2. Trend of total demand compared to dockless', color=['0.25', '0.5', '0.75'])

As seen on the trend above, the trend of dockless electric bikes closely follow the change in trend of all electric bikes. However, dockless electric rides are initially more popular than docking electric rides, from January to April 2021. After that point, dockless electric rides have been less popular than docking electric rides.

The trend started to take shape starting in January 2022, and since January 2023, the trend for all electric rides, which consists of docking and dockless are almost the exact same. Both also follow the trend for electric ride as a whole. This means that probably, after the initial launch of dockless rides, each segment has its own users.

### Question 3 - Comparison of Starting and Ending Stations

We first have to create a new dataframe which consists of the station IDs and the counts of both start and end

In [ ]:
station_counts = (
    pd.concat(
        [rides['start_station_id'].value_counts(dropna=False),
         rides['end_station_id'].value_counts(dropna=False)],
         axis=1,
         keys=['start_count', 'end_count']
    )
    .fillna(0)
    .astype(int)
    .rename_axis('station_id')
    .reset_index()
)

The NaN value, however, is not a string, and therefore will not be searchable. Hence, we convert it into a string

In [ ]:
station_counts['station_id'] = (
    station_counts['station_id']
    .fillna('NaN')
)

station_counts.head()

The top 10 starting station IDs

In [ ]:
starting = station_counts[['station_id', 'start_count']].sort_values('start_count', ascending=False).head(11)
starting

The top 10 starting station IDs

In [ ]:
ending = station_counts[['station_id', 'end_count']].sort_values('end_count', ascending=False).head(11)
ending

In [ ]:
def find_different_count(station_data):
    index = 0
    while(index < len(station_data)):
        start = starting.iloc[index]['station_id']
        end = ending.iloc[index]['station_id']

        if start != end:
            print("Starting and ending stations at number", index+1, "are different")
            print("Start ID =", start, "End ID =", end)
            break

        index+=1


find_different_count(station_counts)

The top 10 are the same if we consider the NaN (dockless) to be a station. Otherwise, the 11th most used station for both starting and ending are different

### Question 4 - Circular Ride Frequency

In [ ]:
df4 = rides
df4['circular'] = (df4['start_station_id'] == df4['end_station_id'])
df4.head(5)

Because the 'circular' column is a boolean, we just sum them. Or we can also not sum them to see the difference between circular and non circular drives.

In [ ]:
df4['circular'].value_counts()

### Question 5 - Very Low Activity Stations

We first find how many days have elapsed between January 1st 2021 and December 31st 2023

In [ ]:
from datetime import date

start_date = date(2021, 1, 1)
end_date = date(2023, 12, 31)
delta = end_date-start_date

print("Total days elapsed", delta.days)

Then we get the stations with total number of visits less than the total days elapsed. This means that the station gets at least less than 2 visits per day, either as a starting or an ending point.

These stations can be considered to be phased out. However, we should look further than just the numbers. For example, if the station is located underneath a hill, then maybe the lack of visit is explainable.

In [ ]:
station_counts.query(f"start_count < {delta.days} | end_count < {delta.days}")

### Question 6 - Seasonal Patterns of Stations

Start by stacking the count of both the start and the end station since we want to see the total station visit

In [ ]:
visits = pd.concat([
    rides[['start_station_id', 'month']].rename(columns={'start_station_id':'station_id'}),
    rides[['end_station_id', 'month']].rename(columns={'end_station_id':'station_id'}),
])

visits['month'] = visits['month'].astype('str')
visits['month'] = visits['month'].str[5:7]
visits['month'] = visits['month'].astype('int64')
visits

Here we create our variation. Our variation uses range, which denotes the difference between the minimum and maximum values

In [ ]:
monthly = (
    visits
        .groupby(['station_id', 'month'])
        .size()
        .unstack(fill_value=0)
)

monthly['range'] = monthly.max(axis=1) - monthly.min(axis=1)

monthly

Now we sort to get the top 5

In [ ]:
top_variation = monthly['range'].sort_values(ascending=False).head(5)
top_variation

Plot of the top 5

In [ ]:
monthly.loc[top_variation.index, list(range(1,13))].T.plot(figsize=(12, 7), ylabel='Number of visits', stacked=False)

Now we try finding the highest jump between month to month of our top 5 stations with the highest range.

In [ ]:
import calendar

for i in range(len(top_variation.index)):
    curr_item = monthly.loc[top_variation.index[i]]
    max_diff = 0
    month_start = 0
    month_end = 1
    for j in range(len(curr_item)-2):
        diff = abs(curr_item.iloc[j] - curr_item.iloc[j+1])
        if (diff > max_diff):
            max_diff = diff
            month_start = j
            month_end = j+1
    print(f"Maximum variation for {curr_item.name} is {max_diff}, between {calendar.month_abbr[month_start]} and {calendar.month_abbr[month_end]}")